# Lab 3: Azure AI Foundry Agent Service - Connected Agent Pattern

## Overview

In this notebook, we will build and deploy a Multi-Agent system using **Azure AI Foundry Agent Service**.

### 🔑 Key Implementation Approach

- **Azure AI Foundry Agent Service**: Using `azure-ai-projects` SDK
- **Connected Agent Pattern**: Connecting Agents via Handoff API
- **Thread-based Conversation**: Agent Service automatically manages context
- **Managed Execution**: Azure provides Agent execution environment

### 💡 Differences from Lab 4

| Characteristic | Lab 3 (This Notebook) | Lab 4 (Workflow Pattern) |
|------|------------------|-------------------------|
| **Agent Foundation** | ✅ Foundry Agent Service | ✅ Foundry Agent Service |
| **Workflow** | **Connected Agent (Handoff)** | **Workflow Pattern (Router+Executor)** |
| **Routing Method** | `handoff_to_agent()` API | Router Executor Function |
| **Execution Flow** | Main → Handoff → Sub Agent | Router → Executor → Output |
| **State Management** | Thread-based | Workflow Context-based |

> **✅ Commonality**: Both Labs use the **same Azure AI Foundry Agent Service**.
> 
> **🎯 Difference**: Different **Agent orchestration patterns** (Connected Agent vs Workflow Pattern).

### Architecture
```
┌────────────────────────────────────────────────────────────┐
│                 Multi-Agent System                         │
│                                                            │
│  ┌─────────────────────────────────────────────┐          │
│  │          Main Agent                         │          │
│  │  (Task Analysis & Routing)                  │          │
│  └────────────┬────────────────┬────────────────┘          │
│               │                │                           │
│       ┌───────▼──────┐  ┌──────▼──────────┐               │
│       │  Tool Agent  │  │  Research       │               │
│       │  (MCP)       │  │  Agent (RAG)    │               │
│       └──────┬───────┘  └────────┬────────┘               │
│              │                   │                         │
│       ┌──────▼───────┐    ┌──────▼─────────┐              │
│       │  MCP Server  │    │  Azure AI      │              │
│       │  (ACA)       │    │  Search (RAG)  │              │
│       └──────────────┘    └────────────────┘              │
└────────────────────────────────────────────────────────────┘
```

### Key Components

1. **Main Agent**: Analyzes user requests and routes to appropriate Agent
2. **Tool Agent**: Utilizes tools from MCP server (real-time weather information)
3. **Research Agent**: Knowledge base search via Azure AI Search
4. **MCP Server**: Tool server deployed on Azure Container Apps

### Python Module Structure

```
src/foundry_agent/
├── main_agent.py       - Main Agent class
├── tool_agent.py       - Tool Agent class (MCP)
├── research_agent.py   - Research Agent class (RAG)
├── api_server.py       - FastAPI HTTP server (Agent endpoint)
├── masking.py          - Masking utility (PII protection)
└── requirements.txt    - Python dependencies
```

### Learning Objectives

1. ✅ Understand and utilize Azure AI Foundry Agent Service
2. ✅ Deploy MCP Server to Azure Container Apps
3. ✅ MCP integration using Connected Agent
4. ✅ Implement Multi-Agent orchestration pattern
5. ✅ Build RAG-based Agent
6. ✅ Agent collaboration and response integration


---

## ⚙️ Before You Start

**Select Python Kernel:**

1. Click **"Select Kernel"** in the top right of the notebook
2. Select **"Python Environments..."**
3. Select **`.venv (Python 3.x.x)`** (virtual environment created in project root)

> 💡 **GitHub Codespaces**: The `.venv` environment is automatically created in Codespaces.  
> If you don't see `.venv`, create it with `python -m venv .venv` in the terminal.

---


## 1. Setup & Authentication

### Tenant ID Configuration Guide

**In most cases**: You don't need to specify a tenant ID. Leave the `tenant_id` variable as `"<YOUR_TENANT_ID>"` or `None` and run.

**When Tenant ID is needed**:
- ✅ When you have access to Azure tenants of multiple organizations (companies)
- ✅ When you need to work only with resources of a specific organization
- ✅ When you get "multiple tenants" related errors during login

**How to find Tenant ID**:
- Azure Portal → Azure Active Directory → Overview → Copy Tenant ID
- Or contact your organization administrator


In [ ]:
import sys, subprocess, os, json
import platform

# Set PATH based on OS
system = platform.system()
if system == 'Darwin':  # macOS
    extra_paths = '/opt/homebrew/bin:/usr/local/bin'
elif system == 'Linux':  # Linux / Codespaces
    extra_paths = '/usr/local/bin:/usr/bin:/home/codespace/.local/bin'
else:  # Windows
    extra_paths = ''

if extra_paths:
    os.environ['PATH'] = extra_paths + ':' + os.environ.get('PATH', '')

def check(cmd, name):
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, timeout=3, env=os.environ)
        print(f"{'✓' if result.returncode == 0 else '✗'} {name}")
    except Exception as e:
        print(f"✗ {name}")

print("=== Prerequisites Check ===")
print(f"✓ Python {sys.version.split()[0]} ({system})")
check("az --version", "Azure CLI")
check("docker --version", "Docker")
print("="*50)


In [ ]:
import subprocess, json

print("=== Azure Authentication ===")
print("ℹ️  Checking authentication status and logging in if necessary.\n")

# Enter your tenant ID here (optional)
# Example: tenant_id = "16b3c013-d300-468d-ac64-7eda0820b6d3"
tenant_id = "<YOUR_TENANT_ID>"  # Or set to None to use default tenant

# Check Azure CLI authentication status
az_account = subprocess.run("az account show", shell=True, capture_output=True, text=True)

if az_account.returncode == 0:
    account_info = json.loads(az_account.stdout)
    print(f"✅ Azure CLI authentication complete (using existing session)")
    print(f"   Subscription: {account_info.get('name', 'N/A')}")
    print(f"   Tenant: {account_info.get('tenantId', 'N/A')}")
else:
    print("⚠️  Azure CLI authentication required. Opening browser...")
    # If tenant ID is set, login with that tenant
    if tenant_id and tenant_id != "<YOUR_TENANT_ID>":
        az_login = subprocess.run(f"az login --tenant {tenant_id}", shell=True)
    else:
        az_login = subprocess.run("az login", shell=True)
    
    if az_login.returncode == 0:
        print("✅ Azure CLI login complete")
    else:
        raise Exception("❌ Azure CLI login failed")

print("="*50)


In [ ]:
# Load configuration file
config_path = "config.json"
with open(config_path) as f:
    config = json.load(f)

# Set environment variables
RESOURCE_GROUP = config["resource_group"]
LOCATION = config["location"]
PROJECT_CONNECTION_STRING = config["project_connection_string"]
SEARCH_ENDPOINT = config["search_endpoint"]
SEARCH_INDEX = config["search_index"]
CONTAINER_REGISTRY = config["container_registry_endpoint"]
CONTAINER_ENV_ID = config["container_apps_environment_id"]

# Convert PROJECT_CONNECTION_STRING to simple format
# config.json format: https://xxx/api/projects/yyy;subscription_id=zzz;resource_group=www
# Required format: https://xxx/api/projects/yyy (remove after semicolon)
simple_project_conn = PROJECT_CONNECTION_STRING.split(';')[0] if PROJECT_CONNECTION_STRING else ""

print("=== Configuration Loaded ===")
print(f"Resource Group: {RESOURCE_GROUP}")
print(f"Location: {LOCATION}")
print(f"Search Index: {SEARCH_INDEX}")
print(f"Container Registry: {CONTAINER_REGISTRY}")
print("="*50)


## 2. Install Required Packages

Install essential Azure AI packages. Most packages may already be installed if running in GitHub Codespaces.


In [ ]:
# Install required packages
import subprocess
import sys

packages = [
    "azure-ai-projects",
    "azure-ai-evaluation",
    "azure-ai-inference",
    "azure-search-documents",
    "azure-identity",
    "openai",
    "python-dotenv",
    "requests"
]

print("=== Installing Required Packages ===\n")

for package in packages:
    print(f"Installing {package}...")
    result = subprocess.run(
        [sys.executable, "-m", "pip", "install", "-q", package],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        print(f"✅ {package} installed")
    else:
        print(f"⚠️  {package} may already be installed or failed to install")

print("\n" + "="*50)
print("✅ Package installation completed!")


## 3. Get Azure AI Search Key

Retrieve the Azure AI Search admin key to be used by the RAG Agent.


In [ ]:
# Get AI Search admin key
search_name = config["search_service_name"]

search_key_cmd = f"""
az search admin-key show \
    --resource-group {RESOURCE_GROUP} \
    --service-name {search_name} \
    --query primaryKey -o tsv
"""

result = subprocess.run(search_key_cmd, shell=True, capture_output=True, text=True)
SEARCH_KEY = result.stdout.strip()

if SEARCH_KEY:
    print(f"✅ Search key retrieved: {SEARCH_KEY[:10]}...")
    os.environ['SEARCH_KEY'] = SEARCH_KEY
else:
    print("❌ Failed to retrieve search key")


## 4. Add Azure AI Search Connection

⚠️ **CRITICAL**: You must add an Azure AI Search connection to your Azure AI Foundry project.

### 🔴 Why Connection is Required

**Without connection:**
- ❌ Research Agent's RAG functionality **will NOT work at all**
- ❌ `AzureAISearchTool` cannot access the search service
- ❌ Agent can only answer with general knowledge, not from knowledge base
- ❌ Citation (source attribution) functionality is unavailable

**With connection:**
- ✅ Research Agent can search knowledge base via Azure AI Search
- ✅ `AzureAISearchTool` automatically uses project connection
- ✅ Provides accurate RAG-based answers
- ✅ Can display answer sources with Citation

### 📋 What This Section Does

In this section:
1. ✅ Check existing Azure AI Search connections
2. ⚠️ Provide manual setup instructions if connection doesn't exist
3. ✅ Verify connection and proceed to next step

### 🔧 How to Set Up Connection

**Run the cell below to check connection status.**

If connection doesn't exist, add it in Azure Portal as follows:

1. **Access Azure AI Foundry Portal**: https://ai.azure.com
2. **Select Project**: Open your current project
3. **Management center → Resource → Connected resources → + New connection**
4. **Select Azure AI Search**
5. **Enter connection information**:
   - Endpoint: `{SEARCH_ENDPOINT}`
   - API Key: `{SEARCH_KEY}` (automatically displayed in cell below)
6. **Save**

> 💡 **Note**: Connection setup is one-time only, and all Agents will automatically use it afterwards.


In [ ]:
# Check Azure AI Search connection
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
from azure.identity import DefaultAzureCredential

print("="*70)
print("🔍 Azure AI Search Connection Check")
print("="*70)
print("\n⚠️  This step is essential for Research Agent's RAG functionality!\n")

# Initialize project client
print(f"📂 Project: {simple_project_conn}\n")

project_client_for_connection = AIProjectClient(
    endpoint=simple_project_conn,
    credential=DefaultAzureCredential()
)

# Check existing connections
print("🔍 Checking existing connections...\n")

connection_exists = False
connection_name = None

try:
    # Try to get default Azure AI Search connection
    search_connection = project_client_for_connection.connections.get_default(
        connection_type=ConnectionType.AZURE_AI_SEARCH
    )
    print(f"✅ Found default Azure AI Search connection!")
    print(f"   • Connection ID: {search_connection.id}")
    print(f"   • Connection Name: {search_connection.name}\n")
    connection_exists = True
    connection_name = search_connection.name
    
except Exception as e:
    print(f"⚠️  Default connection not found: {str(e)[:100]}\n")
    
    # Try to list all Azure AI Search connections
    try:
        print("🔍 Searching for all Azure AI Search connections...")
        connections = list(project_client_for_connection.connections.list(
            connection_type=ConnectionType.AZURE_AI_SEARCH
        ))
        
        if connections:
            print(f"✅ Found {len(connections)} Azure AI Search connection(s):\n")
            for conn in connections:
                print(f"   • {conn.name}")
                print(f"     ID: {conn.id}\n")
            connection_exists = True
            connection_name = connections[0].name
        else:
            print("❌ No Azure AI Search connections found!\n")
            
    except Exception as e2:
        print(f"❌ Failed to list connections: {str(e2)[:100]}\n")

# Display result with enhanced guidance
print("="*70)

if connection_exists:
    print("✅✅✅ Azure AI Search connection configured successfully! ✅✅✅")
    print(f"\n📋 Connection Information:")
    print(f"   • Connection Name: {connection_name}")
    print(f"   • Search Endpoint: {SEARCH_ENDPOINT}")
    print(f"   • Index: {SEARCH_INDEX}")
    print("\n✅ Research Agent will use this connection for RAG.")
    print("✅ Knowledge base search and Citation features will work properly.")
    print("\n💡 Proceed to the next cell!\n")
else:
    print("❌❌❌ No Azure AI Search connection! ❌❌❌")
    print("\n🔴 If you proceed in this state:")
    print("   • Research Agent will only answer with general knowledge, without RAG")
    print("   • Knowledge base search functionality will not work")
    print("   • Citation (source attribution) will be unavailable")
    print("\n📝 To add connection in Azure Portal:")
    print("   1. 🌐 Access https://ai.azure.com")
    print(f"   2. 📂 Select Project: {simple_project_conn.split('/')[-1]}")
    print("   3. ⚙️  Settings → Connections → + New connection")
    print("   4. 🔍 Select Azure AI Search")
    print("   5. 📋 Configure connection with:")
    print(f"      • Endpoint: {SEARCH_ENDPOINT}")
    print(f"      • API Key: {SEARCH_KEY[:10]}... (displayed in section 3 above)")
    print("   6. 💾 Click Save")
    print("\n⚠️  Rerun this cell after adding connection to verify!")
    print("\n📌 Adding connection is required. Please complete before proceeding.\n")

print("="*70)


## 5. Deploy MCP Server

Deploy the Model Context Protocol server to Azure Container Apps.

### MCP Server Features
- `get_weather`: Query real-time weather information by city
  - Data source: wttr.in API (free, highly reliable)
  - Supported languages: Korean/English city names (e.g., 'Seoul', '서울')
  - Provided info: temperature, feels-like, weather condition, humidity, wind speed/direction, observation time

### Server Configuration
- **Protocol**: Streamable HTTP (MCP over HTTP with SSE)
- **Port**: 8000
- **Endpoints**:
  - `POST /mcp` - MCP message handling (Server-Sent Events)


In [ ]:
# Login to Container Registry (for redeployment)
registry_name = CONTAINER_REGISTRY.split('.')[0]

print("=== Container Registry Login ===")
login_cmd = f"az acr login --name {registry_name}"
result = subprocess.run(login_cmd, shell=True, capture_output=True, text=True)

if result.returncode == 0:
    print(f"✅ Logged in to {registry_name}")
    print("\n💡 MCP server has been updated. Rebuild in the next cell.")
else:
    print(f"❌ Login failed: {result.stderr}")
print("="*50)


In [ ]:
# Create .env file (for MCP Server - for future extensibility)
print("=== Creating .env file for MCP Server ===\n")

# MCP server doesn't currently need environment variables, but creating empty file for future expansion
env_content = """# MCP Server Configuration
# Add any configuration variables here as needed

# Example: API keys, endpoints, etc.
# WEATHER_API_KEY=your_api_key_here
# EXTERNAL_SERVICE_URL=your_service_url_here
"""

env_file_path = "src/mcp/.env"

try:
    with open(env_file_path, 'w') as f:
        f.write(env_content)
    
    print(f"✅ Created {env_file_path}")
    print("\n💡 MCP server doesn't currently need environment variables.")
    print("   However, you can add settings to this file for future external API integration.")
    
except Exception as e:
    print(f"❌ Failed to create .env file: {e}")

print("\n" + "="*60)


In [ ]:
# Build and push MCP Server image
import time

mcp_image = f"{CONTAINER_REGISTRY}/mcp-server:latest"

print("=== Building MCP Server Image ===")
print(f"Image: {mcp_image}\n")

# Build (linux/amd64 platform for Azure Container Apps)
build_cmd = f"docker build --platform linux/amd64 -t {mcp_image} ./src/mcp"
print("🔨 Building image (linux/amd64)...")
start_time = time.time()

result = subprocess.run(build_cmd, shell=True, capture_output=True, text=True)
elapsed = time.time() - start_time

if result.returncode == 0:
    print(f"✅ Build successful ({elapsed:.1f}s)")
else:
    print(f"❌ Build failed: {result.stderr}")
    
# Push
if result.returncode == 0:
    print("\n📤 Pushing image to registry...")
    push_cmd = f"docker push {mcp_image}"
    result = subprocess.run(push_cmd, shell=True, capture_output=True, text=True)
    
    if result.returncode == 0:
        print(f"✅ Push successful")
    else:
        print(f"❌ Push failed: {result.stderr}")

print("="*50)


In [ ]:
# Deploy MCP Server as Container App
mcp_app_name = "mcp-server"

print("=== Deploying MCP Server to ACA ===")
print(f"App Name: {mcp_app_name}\n")

deploy_cmd = f"""
az containerapp create \
    --name {mcp_app_name} \
    --resource-group {RESOURCE_GROUP} \
    --environment {CONTAINER_ENV_ID.split('/')[-1]} \
    --image {mcp_image} \
    --target-port 8000 \
    --ingress external \
    --min-replicas 1 \
    --max-replicas 3 \
    --cpu 0.5 \
    --memory 1.0Gi \
    --registry-server {CONTAINER_REGISTRY}
"""

print("🚀 Deploying...")
result = subprocess.run(deploy_cmd, shell=True, capture_output=True, text=True)

if result.returncode == 0:
    print("✅ Deployment successful")
    
    # Get endpoint
    show_cmd = f"""
    az containerapp show \
        --name {mcp_app_name} \
        --resource-group {RESOURCE_GROUP} \
        --query properties.configuration.ingress.fqdn -o tsv
    """
    result = subprocess.run(show_cmd, shell=True, capture_output=True, text=True)
    MCP_ENDPOINT = f"https://{result.stdout.strip()}"
    
    print(f"\n🌐 MCP Endpoint: {MCP_ENDPOINT}")
    
    # Update config
    config['mcp_endpoint'] = MCP_ENDPOINT
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)
    print("✅ Config updated")
else:
    print(f"❌ Deployment failed: {result.stderr}")
    MCP_ENDPOINT = None

print("="*50)


## 6. Build & Deploy Agent Container

Build Agent modules into a container image and deploy to Azure Container Apps.

### Agent Modules Included in Container

- `main_agent.py` - Main Agent class
- `tool_agent.py` - Tool Agent class 
- `research_agent.py` - Research Agent class
- `api_server.py` - FastAPI-based HTTP API server
- `masking.py` - Masking utility (PII protection)

### Server Configuration

- **Framework**: FastAPI
- **Port**: 8000
- **Endpoints**:
  - `/health` - Health check
  - `/` - Root (Agent status information)
  - `/chat` - Agent conversation endpoint (POST)

### MCP Server Features

- **Real-time Weather Information**:
  - `get_weather(location)` - Accurate real-time weather information for cities worldwide
  - Data source: wttr.in API (free, highly reliable)
  - Supported languages: Korean/English city names (e.g., 'Seoul', '서울')
  - Provided info: temperature, feels-like, weather condition, humidity, wind speed/direction, observation time

### Environment Variables

The following variables are automatically configured in the `.env` file:

- `PROJECT_CONNECTION_STRING` - Azure AI Foundry project connection string
- `SEARCH_ENDPOINT`, `SEARCH_KEY`, `SEARCH_INDEX` - Azure AI Search configuration
- `MCP_ENDPOINT` - MCP Server endpoint
- `APPLICATIONINSIGHTS_CONNECTION_STRING` - Application Insights connection (for Application Analytics)
- `OTEL_SERVICE_NAME`, `OTEL_TRACES_EXPORTER`, `OTEL_METRICS_EXPORTER`, `OTEL_LOGS_EXPORTER`, `OTEL_PYTHON_LOGGING_AUTO_INSTRUMENTATION_ENABLED` - OpenTelemetry configuration (for Tracing)
- `AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED` - Enable Prompt/Completion recording (for Tracing UI)
- `AGENT_MASKING_MODE` - Masking intensity setting (off/standard/strict)

### Deployment Architecture

```
┌─────────────────────────────────────────────────────┐
│      Azure Container Apps Environment               │
│                                                     │
│  ┌──────────────────┐  ┌──────────────────────┐   │
│  │  MCP Server      │  │  Agent Service       │   │
│  │  (Weather API)   │  │  (Multi-Agent)       │   │
│  │  :8000           │  │  :8000               │   │
│  │                  │  │                      │   │
│  │  • get_weather() │  │  • Main Agent        │   │
│  │                  │  │  • Tool Agent ────┐  │   │
│  └──────────────────┘  │  • Research Agent │  │   │
│          ▲             │                   │  │   │
│          │             │                   │  │   │
│          └─────────────┼───────────────────┘  │   │
│                        │                      │   │
│                        │  + Azure AI Search   │   │
│                        │  + Azure OpenAI      │   │
│                        └──────────────────────┘   │
└─────────────────────────────────────────────────────┘
```


In [ ]:
# Create .env file (environment variables to be included in Agent Container)
print("=== Creating .env file for Agent Container ===\n")

# 1. Get Application Insights Connection String
print("📊 Getting Application Insights connection string...")
appinsights_cmd = f"""
az monitor app-insights component show \
    --resource-group {RESOURCE_GROUP} \
    --query "[0].connectionString" -o tsv
"""

result = subprocess.run(appinsights_cmd, shell=True, capture_output=True, text=True)

if result.returncode == 0 and result.stdout.strip():
    APP_INSIGHTS_CONN_STR = result.stdout.strip()
    print(f"✅ Application Insights connection string retrieved\n")
else:
    print(f"⚠️  Could not get Application Insights connection string")
    print(f"   Error: {result.stderr}")
    print(f"   Proceeding without Application Insights (Analytics will not work)\n")
    APP_INSIGHTS_CONN_STR = ""

# 2. Get model configuration (from config.json)
model_deployment_name = config.get("model_deployment_name", "gpt-4o")
model_version = config.get("model_version", "2024-11-20")
model_capacity = config.get("model_capacity", 50)
print(f"📦 Model Configuration:")
print(f"   Deployment Name: {model_deployment_name}")
print(f"   Model Version: {model_version}")
print(f"   Capacity (TPM): {model_capacity}")
print(f"   (from config.json - set in Lab 1 infrastructure deployment)\n")

# 3. Create .env file
"""
NOTE: Adding AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED=true enables
input/output (prompt, completion) recording in Tracing UI.
AGENT_MASKING_MODE controls prompt/response masking intensity (off|standard|strict).
"""
env_content = f"""# Azure AI Foundry Configuration
PROJECT_CONNECTION_STRING={simple_project_conn}

# Model Configuration
# The model deployment name and version from Azure OpenAI
# These values are automatically set from Lab 1 infrastructure deployment (infra/main.bicep)
AZURE_AI_MODEL_DEPLOYMENT_NAME={model_deployment_name}
AZURE_AI_MODEL_VERSION={model_version}

# Azure AI Search Configuration
SEARCH_ENDPOINT={SEARCH_ENDPOINT}
SEARCH_KEY={SEARCH_KEY}
SEARCH_INDEX={SEARCH_INDEX}

# MCP Server Configuration
MCP_ENDPOINT={MCP_ENDPOINT if MCP_ENDPOINT else ''}

# Application Insights Configuration (for Application Analytics)
APPLICATIONINSIGHTS_CONNECTION_STRING={APP_INSIGHTS_CONN_STR}

# OpenTelemetry Configuration (for Tracing)
OTEL_SERVICE_NAME=azure-ai-agent
OTEL_TRACES_EXPORTER=azure_monitor
OTEL_METRICS_EXPORTER=azure_monitor
OTEL_LOGS_EXPORTER=azure_monitor
OTEL_PYTHON_LOGGING_AUTO_INSTRUMENTATION_ENABLED=true
AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED=true  # Enable GenAI content capture for tracing UI

# Masking / PII Handling
AGENT_MASKING_MODE=standard  # off|standard|strict (span prompt/completion masking mode)
"""

env_file_path = "src/foundry_agent/.env"

try:
    with open(env_file_path, 'w') as f:
        f.write(env_content)
    
    print(f"✅ Created {env_file_path}")
    print("\n📋 Environment variables:")
    for line in env_content.strip().split('\n'):
        if line and not line.startswith('#'):
            key = line.split('=')[0]
            print(f"   • {key}")
    
    print("\n💡 This file will be included in the Docker image.")
    print("   No separate environment variable configuration needed during deployment.")
    
    if APP_INSIGHTS_CONN_STR:
        print("\n✅ Application Insights configuration complete!")
        print("   → You can check metrics in Application Analytics.")
        print("   → OpenTelemetry Tracing configuration complete!")
        print("   → GenAI content recording enabled (attempting to record Input/Output)")
    else:
        print("\n⚠️  Application Insights not configured")
        print("   → Application Analytics will not work, but Agent will function normally.")
    
    print("\n🔐 Masking Mode: standard (adjustable via AGENT_MASKING_MODE variable)")
    print(f"🤖 Model: {model_deployment_name} (version {model_version}, capacity {model_capacity}K TPM)")
except Exception as e:
    print(f"❌ Failed to create .env file: {e}")

print("\n" + "="*60)


In [ ]:
# Build and push Agent Container image
import time

agent_image = f"{CONTAINER_REGISTRY}/agent-service:latest"

print("=== Building Agent Service Image ===")
print(f"Image: {agent_image}\n")

# Build (linux/amd64 platform for Azure Container Apps)
build_cmd = f"docker build --platform linux/amd64 -t {agent_image} ./src/foundry_agent"
print("🔨 Building image (linux/amd64)...")
start_time = time.time()

result = subprocess.run(build_cmd, shell=True, capture_output=True, text=True)
elapsed = time.time() - start_time

if result.returncode == 0:
    print(f"✅ Build successful ({elapsed:.1f}s)")
    print(f"   Image contains: main_agent.py, tool_agent.py, research_agent.py")
else:
    print(f"❌ Build failed: {result.stderr}")
    
# Push
if result.returncode == 0:
    print("\n📤 Pushing image to registry...")
    push_cmd = f"docker push {agent_image}"
    result = subprocess.run(push_cmd, shell=True, capture_output=True, text=True)
    
    if result.returncode == 0:
        print(f"✅ Push successful")
    else:
        print(f"❌ Push failed: {result.stderr}")

print("="*50)


## 7. Verify Azure Resources

Verify required Azure resources before deploying Agent Service.

**Verification items:**
- ✅ Azure AI Project resource ID
- ✅ AI Services (Cognitive Services) resource ID

This information will be used in the next deployment step to automatically grant permissions to Managed Identity.


In [ ]:
# Verify Azure AI Project and AI Services resource IDs
print("=== Verifying Azure Resources ===\n")

# 1. Get information from config.json (already loaded)
# PROJECT_CONNECTION_STRING in config.json is already in simple format:
# https://xxx.services.ai.azure.com/api/projects/yyy

# Extract project_name from URL
if '/api/projects/' in simple_project_conn:
    project_name = simple_project_conn.split('/api/projects/')[-1].strip()
else:
    project_name = None

print(f"📋 Project Information:")
print(f"   Resource Group: {RESOURCE_GROUP}")
print(f"   Project Name: {project_name if project_name else 'Not found in connection string'}\n")

# 2. Get AI Project resource ID
# Azure AI Foundry Project is of type Microsoft.CognitiveServices/accounts/projects
print("🔍 Finding AI Project resource...")
if project_name:
    # Search for resources containing project_name
    ai_project_cmd = f"""
    az resource list \
        --resource-group {RESOURCE_GROUP} \
        --query "[?contains(name, '{project_name}') && type=='Microsoft.CognitiveServices/accounts/projects'].id" -o tsv
    """
else:
    # Search by type only (first result)
    ai_project_cmd = f"""
    az resource list \
        --resource-group {RESOURCE_GROUP} \
        --query "[?type=='Microsoft.CognitiveServices/accounts/projects'].id | [0]" -o tsv
    """

result = subprocess.run(ai_project_cmd, shell=True, capture_output=True, text=True)
if result.returncode == 0 and result.stdout.strip():
    ai_project_resource_id = result.stdout.strip()
    print(f"   ✅ AI Project Resource ID:")
    print(f"   {ai_project_resource_id}\n")
else:
    print(f"   ❌ Could not find AI Project")
    print(f"   Error: {result.stderr}\n")
    raise Exception("AI Project not found")

# 3. Get AI Services resource ID (Cognitive Services account)
print("🔍 Finding AI Services (Cognitive Services) resource...")
ai_services_cmd = f"""
az resource list \
    --resource-group {RESOURCE_GROUP} \
    --resource-type Microsoft.CognitiveServices/accounts \
    --query "[0].id" -o tsv
"""

result = subprocess.run(ai_services_cmd, shell=True, capture_output=True, text=True)
if result.returncode == 0 and result.stdout.strip():
    ai_services_resource_id = result.stdout.strip()
    print(f"   ✅ AI Services Resource ID:")
    print(f"   {ai_services_resource_id}\n")
else:
    print(f"   ❌ Could not find AI Services")
    print(f"   Error: {result.stderr}\n")
    raise Exception("AI Services not found")

print("✅ All required resources verified!")
print("\n💡 Permissions will be granted to these resources in the next step.")
print("="*60)


## 8. Deploy Agent Service with Permissions

Deploy Agent Service and **automatically configure** Managed Identity immediately after deployment.

### Automated Tasks

1. ✅ Deploy Container App
2. ✅ Enable System-assigned Managed Identity
3. ✅ Assign Azure AI User role (AI Project scope) ← agents/write permission
4. ✅ Wait for permission propagation and restart Container

> 💡 **Important**: Takes approximately 3-4 minutes to complete as deployment and permission setup are handled together.
> 
> ⚠️ **Note**: Managed Identity can only be enabled after Container App is created, so permissions are set immediately after deployment.


In [ ]:
# Deploy Agent Service as Container App + Managed Identity permission setup
agent_app_name = "agent-service"

print("=== Deploying Agent Service to ACA ===")
print(f"App Name: {agent_app_name}\n")

print("💡 Environment variables are already included in the Docker image.")
print("   No separate environment variable configuration needed.\n")

# 1. Deploy Container App (with Managed Identity, replicas 0 until permissions granted)
deploy_cmd = f"""
az containerapp create \
    --name {agent_app_name} \
    --resource-group {RESOURCE_GROUP} \
    --environment {CONTAINER_ENV_ID.split('/')[-1]} \
    --image {agent_image} \
    --target-port 8000 \
    --ingress external \
    --min-replicas 0 \
    --max-replicas 3 \
    --cpu 1.0 \
    --memory 2.0Gi \
    --registry-server {CONTAINER_REGISTRY} \
    --system-assigned \
"""

print("🚀 Deploying Agent Service with Managed Identity...")
print("   (Starting with 0 replicas to configure permissions first)")
result = subprocess.run(deploy_cmd, shell=True, capture_output=True, text=True, timeout=180)

if result.returncode == 0:
    print("✅ Deployment successful\n")
    
    # Get endpoint
    show_cmd = f"""
    az containerapp show \
        --name {agent_app_name} \
        --resource-group {RESOURCE_GROUP} \
        --query properties.configuration.ingress.fqdn -o tsv
    """
    result = subprocess.run(show_cmd, shell=True, capture_output=True, text=True)
    AGENT_ENDPOINT = f"https://{result.stdout.strip()}"
    
    print(f"🌐 Agent Endpoint: {AGENT_ENDPOINT}")
    
    # Update config
    config['agent_endpoint'] = AGENT_ENDPOINT
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)
    print("✅ Config updated\n")
    
    # 2. Get Managed Identity Principal ID
    print("="*60)
    print("🔐 Configuring Permissions\n")
    
    print("1️⃣ Getting Managed Identity Principal ID...")
    identity_cmd = f"""
    az containerapp show \
        --name {agent_app_name} \
        --resource-group {RESOURCE_GROUP} \
        --query identity.principalId -o tsv
    """
    
    result = subprocess.run(identity_cmd, shell=True, capture_output=True, text=True)
    if result.returncode == 0 and result.stdout.strip():
        principal_id = result.stdout.strip()
        print(f"   ✅ Principal ID: {principal_id}\n")
    else:
        print(f"   ❌ Failed to get Principal ID: {result.stderr}\n")
        raise Exception("Failed to get Managed Identity Principal ID")
    
    # 3. Assign Azure AI User role (AI Project scope - for agents/write permission)
    print("4️⃣ Assigning 'Azure AI User' role to AI Project...")
    print(f"   Scope: {ai_project_resource_id}")
    role_assignment_cmd = f"""
    az role assignment create \
        --assignee {principal_id} \
        --role "Azure AI User" \
        --scope {ai_project_resource_id}
    """
    
    result = subprocess.run(role_assignment_cmd, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print("   ✅ Azure AI User role assigned (AI Project scope)\n")
    elif "already exists" in result.stderr.lower():
        print("   ✅ Azure AI User role already exists (AI Project scope)\n")
    else:
        print(f"   ❌ Role assignment FAILED!")
        print(f"   Error: {result.stderr}")
        print(f"   Return code: {result.returncode}\n")
    
    # 4. Verify role assignments
    print("5️⃣ Verifying role assignments...\n")
    import time
    time.sleep(5)  # Brief wait (confirm role assignment API completion)
    
    role_check_cmd = f"""
    az role assignment list \
        --assignee {principal_id} \
        --query "[].{{role:roleDefinitionName, scope:scope}}" -o json
    """
    
    result = subprocess.run(role_check_cmd, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        import json as json_lib
        current_roles = json_lib.loads(result.stdout)
        
        print(f"   📋 Current Role Assignments ({len(current_roles)} total):\n")
        
        # Check required roles
        required_roles = {
            "Azure AI User (AI Project)": False
        }
        
        for role in current_roles:
            scope_parts = role['scope'].split('/')
            resource_name = scope_parts[-1] if scope_parts else 'Unknown'
            role_name = role['role']
            
            print(f"      • {role_name} → {resource_name}")
            
            if role_name == "Azure AI User":
                if "projects" in role['scope'] or ai_project_resource_id in role['scope']:
                    required_roles["Azure AI User (AI Project)"] = True
        
        print(f"\n   🔍 Required Roles Verification:")
        all_roles_ok = True
        for role_name, assigned in required_roles.items():
            status = "✅" if assigned else "❌"
            print(f"      {status} {role_name}")
            if not assigned:
                all_roles_ok = False
        
        if all_roles_ok:
            print(f"\n   ✅ All required roles are assigned!\n")
        else:
            print(f"\n   ❌ Some required roles are missing!")
            print(f"      If this issue occurs, manually verify permissions in Azure Portal.\n")
    else:
        print(f"   ⚠️  Could not verify roles: {result.stderr}\n")
    
    # 5. Wait for permission propagation notice
    print("="*60)
    print("6️⃣ Permissions assigned - waiting for propagation...\n")
    print("⚠️  Azure RBAC permission propagation can take up to 5-10 minutes.")
    print("   Container will remain in replicas=0 state.\n")
    
    print("📋 Next Steps:")
    print("   1. Verify that all 'Required Roles Verification' above show ✅")
    print("   2. Wait about 2-3 minutes")
    print("   3. Run the cell below (Section 9) to start the Container")
    print("   4. If permission errors still occur:")
    print("      → Wait an additional 2-3 minutes and rerun Section 9\n")
    
    print(f"💡 Principal ID (for permission verification): {principal_id}\n")
    
    print("="*60)
    print("✅ Permissions configured successfully!")
    print(f"\n🌐 Endpoint: {AGENT_ENDPOINT}")
    print(f"\n📋 Assigned Roles:")
    print(f"   • Azure AI User (AI Project scope) ← agents/write permission")
    print(f"\n⏳ Wait for permission propagation before running next cell!")
else:
    print(f"❌ Deployment failed: {result.stderr}")
    AGENT_ENDPOINT = None
print("\n" + "="*60)


## 9. Start Agent Service

After permission propagation is complete, run this cell to start the Container.

**When to run:**
- ⏰ Wait **2-3 minutes** after completing Section 8
- ⚠️ If permission errors occur: Wait an additional 2-3 minutes and rerun

**Tasks performed:**
- ✅ Scale Container App to replicas=1
- ✅ Start Container and check status


In [ ]:
# Scale to 1 replica
scale_cmd = f"""
az containerapp update \
    --name agent-service \
    --resource-group {RESOURCE_GROUP} \
    --min-replicas 1 \
    --max-replicas 1
"""

print("🚀 Scaling to 1 replica...")
result = subprocess.run(scale_cmd, shell=True, capture_output=True, text=True, timeout=120)

if result.returncode == 0:
    print("✅ Agent Service started successfully!")
    print(f"\n🌐 Endpoint: {AGENT_ENDPOINT}")
    print("\n💡 Container startup takes approximately 30 seconds.")
    print(f"   Check logs: az containerapp logs show --name agent-service --resource-group {RESOURCE_GROUP} --tail 50")
else:
    print(f"❌ Failed to start: {result.stderr}")

print("\n" + "="*60)


## 10. 🚀 Test Deployed Agent via HTTP

**Important Distinction:**

**Creating Agent locally in notebook** ❌ Local testing → **Data does NOT appear** in Application Analytics

This section calls the **deployed Container's HTTP API**.
- ✅ Container testing → **Data APPEARS** in Application Analytics
- ✅ Container testing → **Tracing** is also enabled (detailed execution logs)

**Why the difference?**
- Application Analytics only tracks **Agents running in Azure AI Foundry Project**
- Notebook is a local environment, so it's tracked separately
- Agents inside Container are connected to Project and automatically tracked

**Test Method:**
1. Call the deployed Agent Service's `/chat` endpoint
2. Test Agent with various questions
3. Check Application Analytics after 5-10 minutes
4. Check detailed execution flow in **Tracing** tab (LLM requests, Tool calls, etc.)


In [ ]:
import requests
import json

print("=== Testing Deployed Agent Service ===\n")

# Check Agent endpoint
if not AGENT_ENDPOINT:
    print("❌ AGENT_ENDPOINT not configured!")
    print("   Run Sections 8 and 9 first.\n")
else:
    print(f"🌐 Agent Endpoint: {AGENT_ENDPOINT}\n")
    
    # 1. Health check
    print("1️⃣ Health Check:")
    try:
        response = requests.get(f"{AGENT_ENDPOINT}/health", timeout=10)
        if response.status_code == 200:
            print(f"   ✅ Health: {response.json()}\n")
        else:
            print(f"   ❌ Health check failed: {response.status_code}\n")
    except Exception as e:
        print(f"   ❌ Error: {e}\n")
    
    # 2. Check root endpoint (Agent status)
    print("2️⃣ Agent Status:")
    try:
        response = requests.get(f"{AGENT_ENDPOINT}/", timeout=10)
        if response.status_code == 200:
            status = response.json()
            print(f"   ✅ Status: {status.get('status')}")
            print(f"   📋 Agents:")
            for agent_name, available in status.get('agents', {}).items():
                icon = "✅" if available else "❌"
                print(f"      {icon} {agent_name}: {available}")
            print()
        else:
            print(f"   ❌ Status check failed: {response.status_code}\n")
    except Exception as e:
        print(f"   ❌ Error: {e}\n")
    
    print("="*70)
    print("\n💡 Agent Service is running normally!")
    print("   Test with actual questions in the next cell.\n")
    print("="*70)


## 11. Test Main Agent (Various Questions)

Send various questions to the deployed Main Agent to generate Application Analytics data.

### 📚 Research Agent's Citation Feature

When Research Agent uses Azure AI Search to answer, it **automatically displays sources (citations)**:

**Citation Format:**
- `【3:0†source】` = Message 3's 0th annotation
- `【3:1†source】` = Message 3's 1st annotation
- `【4:0†source】` = Message 4's 0th annotation

**Example Response:**
```
Jeju Island's Udo is known for its beautiful natural scenery【3:0†source】.
Seongsan Ilchulbong is listed as a UNESCO World Natural Heritage site【3:1†source】.
```

**How it works:**
1. `AzureAISearchTool` searches knowledge base
2. Found documents automatically injected into Agent context
3. LLM references documents when generating answer
4. Azure AI Foundry SDK automatically adds citations
5. Click each citation in Tracing UI to view original document

> 💡 **Note:** Citation is a built-in feature of Azure AI Foundry Agent Service, automatically generated without separate implementation in code.


In [ ]:
# Send various questions to deployed Main Agent
import requests
import json
import time
import statistics

print("=== Testing Deployed Main Agent ===\n")

# Test cases (3 cases)
test_cases = [
    {
        "message": "Hello.",
        "description": "General conversation"
    },
    {
        "message": "Tell me the current weather in Seoul. Please include temperature, feels-like temperature, weather condition, humidity, and wind information.",
        "description": "Seoul detailed weather information (using Tool Agent)"
    },
    {
        "message": "Recommend tourist attractions in Jeju Island",
        "description": "Travel recommendations (using Research Agent)"
    }
]

# Output options
SHOW_FULL_RESPONSE = True  # Set to False to show only first 200 characters
PREVIEW_CHARS = 200

success_count = 0
fail_count = 0
latencies = []

for i, test in enumerate(test_cases, 1):
    print(f"\n[{i}/{len(test_cases)}] {test['description']}")
    print(f"Question: {test['message']}")
    
    try:
        start_req = time.perf_counter()
        response = requests.post(
            f"{AGENT_ENDPOINT}/chat",
            json={"message": test['message']},
            headers={"Content-Type": "application/json"},
            timeout=90
        )
        elapsed_req = (time.perf_counter() - start_req) * 1000  # ms
        
        if response.status_code == 200:
            result = response.json()
            full_resp = result.get('response', 'No response') or ''
            latencies.append(elapsed_req)
            
            print(f"✅ Success ({elapsed_req:.0f}ms)")
            if SHOW_FULL_RESPONSE:
                print(f"Response: {full_resp}")
            else:
                preview = full_resp[:PREVIEW_CHARS] + ('...' if len(full_resp) > PREVIEW_CHARS else '')
                print(f"Response: {preview}")
            success_count += 1
        else:
            print(f"❌ Failed (HTTP {response.status_code}): {response.text[:100]}")
            fail_count += 1
        
    except Exception as e:
        print(f"❌ Error: {e}")
        fail_count += 1
    
    if i < len(test_cases):
        time.sleep(2)

print(f"\n{'='*70}")
print(f"📊 Results: ✅ {success_count} successful, ❌ {fail_count} failed")

if latencies:
    avg_latency = statistics.mean(latencies)
    print(f"⏱️  Average response time: {avg_latency:.0f}ms")

if success_count > 0:
    print(f"\n💡 Next steps:")
    print(f"   • Check Analytics in Azure AI Foundry Portal after 5-10 minutes")
    print(f"   • https://ai.azure.com → Project → Evaluation → Application Analytics")
else:
    print(f"\n💡 Troubleshooting:")
    print(f"   • Check Container logs: az containerapp logs show --name agent-service -g {RESOURCE_GROUP} --tail 50")

print("="*70)


## 📍 Next Steps

You have completed Foundry Agent deployment! Now proceed to the next notebooks in order:

1. **Notebook 04**: Deploy MAF-based Agent (`04_deploy_foundry_agent_with_maf.ipynb`)
2. **Notebook 05**: MAF Workflow Patterns Practice (`05_maf_workflow_patterns.ipynb`)
3. **Notebook 06**: MAF Dev UI Practice (`06_maf_dev_ui.ipynb`)
4. **Notebook 07**: Agent Evaluation (`07_evaluate_agents.ipynb`)
